In [1]:
import json
import uuid
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from kafka import KafkaProducer
from faker import Faker
from time import sleep

In [2]:
dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
kafka_host = os.getenv('KAFKA_HOST')
kafka_topic = os.getenv('KAFKA_TOPIC_NAME')
kafka_topic_partition = os.getenv('KAFKA_TOPIC_NAME')

In [4]:
print(kafka_host)
print(kafka_topic)
print(kafka_topic_partition)
print(dotenv_path)
print(os.environ.get('KAFKA_TOPIC_NAME'))

dataeng-kafka
apotek-dibimbing-ajinusa2
apotek-dibimbing-ajinusa2
/resources/.env
apotek-dibimbing-ajinusa2


In [5]:
producer = KafkaProducer(bootstrap_servers=f'{kafka_host}:9092')

In [6]:
_instance = Faker()
global faker
faker = Faker()

In [7]:
from datetime import datetime, timedelta
# DataGenerator untuk penjualan obat
class DataGenerator(object):
    @staticmethod
    def get_data():
        transaction_date = faker.date_this_year()  # Mendapatkan tanggal transaksi
        transaction_time = faker.time()  # Mendapatkan waktu transaksi
        now = datetime.now()
        
        # Mengonversi tanggal dan waktu menjadi string dalam format yang dapat diterima JSON
        transaction_date_str = transaction_date.strftime('%Y-%m-%d') if isinstance(transaction_date, datetime) else str(transaction_date)
        transaction_time_str = transaction_time  # Faker time() sudah berupa string (jam:menit:detik)
        
         # Data transaksi
        quantity = faker.random_int(min=1, max=5)  # Jumlah obat yang dibeli
        unit_price = faker.random_int(min=1000, max=20000)  # Harga per unit obat (dalam IDR)
        total_price = quantity * unit_price  # Total harga transaksi

        return [
            uuid.uuid4().__str__(),  # Transaction ID
            faker.name(),  # Nama pembeli
            faker.random_element(elements=('Paracetamol', 'Ibuprofen', 'Amoxicillin', 'Vitamin C', 'Antibiotik', 'Cough Syrup')),  # Nama obat
            quantity,  # Jumlah obat yang dibeli
            unit_price,  # Harga per unit obat (dalam IDR)
            total_price,  # Total harga transaksi
            faker.random_element(elements=('Cash', 'Credit Card', 'Debit Card', 'QRIS')),  # Metode pembayaran
            transaction_date_str,  # Tanggal transaksi sebagai string
            transaction_time_str,  # Waktu transaksi sebagai string
            faker.unix_time(
                start_datetime=now - timedelta(minutes=60), end_datetime=now
            ),
        ]

# Mengirimkan data ke Kafka
for i in range(1, 500):  # Jumlah data yang akan dibuat
    columns = ["transaction_id", "buyer_name", "medication_name", "quantity", "unit_price", "total_price", "payment_method", "transaction_date", "transaction_time","ts"]
    data_list = DataGenerator.get_data()
    json_data = dict(zip(columns, data_list))
    _payload = json.dumps(json_data).encode("utf-8")
    response = producer.send(topic=kafka_topic_partition, value=_payload)
    print(json_data['transaction_id'], response.get())  # Menampilkan ID transaksi dan status pengiriman
    sleep(5)  # Delay 5 detik antara setiap pengiriman


77c60592-5a72-42d5-a300-5ca7a791a6b4 RecordMetadata(topic='apotek-dibimbing-ajinusa2', partition=4, topic_partition=TopicPartition(topic='apotek-dibimbing-ajinusa2', partition=4), offset=205, timestamp=1737861572138, log_start_offset=151, checksum=None, serialized_key_size=-1, serialized_value_size=290, serialized_header_size=-1)
2ca30cd8-52e1-408e-a4ed-8d55c1fe9f6f RecordMetadata(topic='apotek-dibimbing-ajinusa2', partition=1, topic_partition=TopicPartition(topic='apotek-dibimbing-ajinusa2', partition=1), offset=165, timestamp=1737861577148, log_start_offset=127, checksum=None, serialized_key_size=-1, serialized_value_size=284, serialized_header_size=-1)
21e363e4-22dc-4913-85c3-bbe664c8697a RecordMetadata(topic='apotek-dibimbing-ajinusa2', partition=0, topic_partition=TopicPartition(topic='apotek-dibimbing-ajinusa2', partition=0), offset=167, timestamp=1737861582159, log_start_offset=130, checksum=None, serialized_key_size=-1, serialized_value_size=289, serialized_header_size=-1)
bec0